## Setup

In [1]:
import numpy as np
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.
/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
x_train = np.expand_dims(x_train, 1)
x_test = np.expand_dims(x_test, 1)
(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

((60000, 1, 28, 28), (60000,), (10000, 1, 28, 28), (10000,))

In [3]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [4]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [6]:
mean_px = x_train.mean().astype(np.float32)
std_px = x_train.std().astype(np.float32)
def norm_input(x): return (x - mean_px) / std_px

## Linear model

In [7]:
def get_lin_model_1():
    model = Sequential([
        Lambda(norm_input, input_shape = (1, 28, 28)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
def get_lin_model():
    model = Sequential([
        Flatten(input_shape = (1, 28, 28)),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [19]:
lm = get_lin_model_1()

In [20]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(x_train, y_train, batch_size=batch_size)
test_batches = gen.flow(x_test, y_test, batch_size=batch_size)

/root/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (60000, 1, 28, 28) (28 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')
/root/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (10000, 1, 28, 28) (28 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


经测试samplewise_std_normalization=True时，与使用norm_input效果类似，但是运行效率差4倍

In [21]:
lm.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
                 validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/1
938/937 [==============================] - 4s 4ms/step - loss: 0.4191 - acc: 0.8764 - val_loss: 0.2922 - val_acc: 0.9181


In [11]:
lm.optimizer.lr = 0.1

In [12]:
lm.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
                 validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/1
938/937 [==============================] - 4s 4ms/step - loss: 5.6961 - acc: 0.6433 - val_loss: 5.6258 - val_acc: 0.6484


In [13]:
lm.optimizer.lr = 0.01

In [14]:
lm.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=4,
                 validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/4
938/937 [==============================] - 4s 4ms/step - loss: 5.6072 - acc: 0.6496 - val_loss: 5.5673 - val_acc: 0.6521
Epoch 2/4
938/937 [==============================] - 4s 4ms/step - loss: 5.5447 - acc: 0.6538 - val_loss: 5.7376 - val_acc: 0.6426
Epoch 3/4
938/937 [==============================] - 4s 4ms/step - loss: 5.5157 - acc: 0.6560 - val_loss: 5.4424 - val_acc: 0.6610
Epoch 4/4
938/937 [==============================] - 4s 4ms/step - loss: 5.5385 - acc: 0.6547 - val_loss: 5.6341 - val_acc: 0.6488
